In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from playwright.async_api import async_playwright

import nltk
from nltk.corpus import stopwords
from nltk.corpus import words as english_words
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

## Casino List
- https://www.casinoslists.com/casinos/best
- https://www.casinofreak.com/reviews/all-online-casinos
- https://casinoalpha.com/online-casinos/list/page/2/
- https://www.nonstopbonus.com/new-online-casinos
- https://list.casino/
- https://lcb.org/casinos

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.5'
}

In [3]:
df_pos = pd.read_csv('data/positives_clean.csv', lineterminator='\n')
df_neg = pd.read_csv('data/negatives_clean.csv', lineterminator='\n')

df_pos['label'] = 1
df_neg['label'] = 0

df = pd.concat([df_pos, df_neg], ignore_index=True)
df

,text,label
0,SISportsbook | Sports Betting Odds | Bet $10 G...,1
1,Online Sportsbook | Bet Online | Online Sports...,1
2,FanDuel | Daily Fantasy Sports and Online U.S....,1
3,To view this video please enable JavaScript an...,1
4,web browser that,1
...,...,...
3417,Free Podcast Hosting - Buzzsprout Buzzsprout m...,0
3418,ORCID Please enable JavaScript to continue usi...,0
3419,Discover Top Blogs & Best Websites in 2023 - F...,0
3420,Google AR & VR | Home OverviewNews & Announcem...,0


In [4]:
print("len", len(df))
df['text'] = df['text'].apply(lambda x: x.lower())
df.loc[:, 'word_count'] = df['text'].str.split().str.len()
df = df[df['word_count'] >= 30]
df = df.drop(columns=['word_count'])
print("len", len(df))
df

len 3422
len 1070


,text,label
0,sisportsbook | sports betting odds | bet $10 g...,1
1,online sportsbook | bet online | online sports...,1
2,fanduel | daily fantasy sports and online u.s....,1
5,supports html5 videoskipnew customersbet $5 ge...,1
6,malta sunsetsign inbeginning on 22 january 202...,1
...,...,...
3416,online community for artists [pixiv] pixiv has...,0
3417,free podcast hosting - buzzsprout buzzsprout m...,0
3419,discover top blogs & best websites in 2023 - f...,0
3420,google ar & vr | home overviewnews & announcem...,0


In [7]:
from playwright.async_api import async_playwright
import asyncio

async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto('https://casino.regevson.com')
        print(await page.title())
        await browser.close()

await run()

Casino Blocker


In [2]:

file_urls = 'data/blocklist/all_urls_pos.txt'
file_output = 'data/blocklist/pos_raw.txt'
dead_urls_output = 'data/blocklist/dead_urls_pos.txt'

def write_dead(url):
    with open(dead_urls_output, "a") as out_file:
        out_file.write(url+"\n")

TIMEOUT = 10
SAVE_THRES = 1
WORD_THRES = 50
async def run():
    contents = []
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        context = await browser.new_context()
        page = await context.new_page()
        await page.set_extra_http_headers({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
            'Accept-Language': 'en-US,en;q=0.5'
        })

        with open(file_urls, 'r') as file:
            urls = file.readlines()
        
        # 780
        # 761
        # 32
        i = 2121
        urls = urls[2121:]
        for url in tqdm(urls, desc="Fetching URLs"):
            url = url.strip() # Remove whitespace/newlines

            # fetch content 
            content = ""
            try:
                await page.goto("https://"+url, timeout=TIMEOUT*1000)
                content = await page.content()
                soup = BeautifulSoup(content, "html.parser")
                content = soup.get_text(strip=True)
            except Exception as e:
                write_dead(url)
                print(f"Error fetching {url}: {e}")
                continue
            
            if content and len(content.split()) > WORD_THRES:
                content = re.sub(r'[^\w .,;:!?"\'-]', ' ', content, flags=re.UNICODE)
                content = re.sub(r'\s+', ' ', content) # compress multiple spaces into one
                content = content.strip()
                contents.append(content)
                i += 1
            
                if i % SAVE_THRES == 0:
                    with open(file_output, "a") as out_file:
                        out_file.write("\n\n".join(contents) + "\n\n")
                    contents = []
            else:
                write_dead(url)


        await context.close()
        await browser.close()

await run()

Fetching URLs:   0%|          | 1/487 [00:10<1:21:18, 10.04s/it]

Error fetching www.lotto-berlin.de: Timeout 10000ms exceeded.


Fetching URLs:   0%|          | 2/487 [00:20<1:21:13, 10.05s/it]

Error fetching www.lotto-brandenburg.de: Timeout 10000ms exceeded.


Fetching URLs:   1%|          | 3/487 [00:30<1:21:03, 10.05s/it]

...

Fetching URLs:  99%|█████████▉| 483/487 [25:50<00:07,  1.87s/it]

Error fetching zodiacu.com: net::ERR_CONNECTION_CLOSED at https://zodiacu.com/


Fetching URLs:  99%|█████████▉| 484/487 [25:52<00:05,  1.89s/it]

Error fetching zpisportbk.com: net::ERR_NAME_NOT_RESOLVED at https://zpisportbk.com/


Fetching URLs: 100%|██████████| 487/487 [26:05<00:00,  3.21s/it]


In [3]:
# remove dead urls
INPUT_ALL = "data/blocklist/all_urls_pos.txt"
INPUT_DEAD = "data/blocklist/dead_urls_pos.txt"
OUTPUT = "data/blocklist/alive_urls_pos.txt"


with open(INPUT_ALL, "r") as input_file:
    all_urls = input_file.readlines()

with open(INPUT_DEAD, "r") as dead_file:
    dead_urls = dead_file.readlines()



alive = list(set(all_urls).difference(set(dead_urls)))

print(len(all_urls))
print(len(dead_urls))
print(len(alive))

with open(OUTPUT, "w") as out_file:
    out_file.write("".join(alive))


2608
1734
1045


In [ ]:
with open('data/blocklist/.txt', 'r') as file:
    lines = file.read().splitlines()  # splitlines() removes the newline character
    lines = [line for line in lines if line.strip()]  # this will remove empty lines